## Volcanic Rock Voucher Trading Strategy

This notebook implements a trading strategy for Volcanic Rock Vouchers, including:

1. Data loading and preprocessing
2. Option pricing models
3. Market making strategy
4. Arbitrage strategy
5. Delta hedging strategy
6. Position management
7. Performance backtesting

The strategy can be tested on historical data and fine-tuned for optimal performance.


In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import os
from datetime import datetime
import time

In [15]:
# Global configuration parameters
CONFIG = {
    'days_to_expiry': 7,        # Days until vouchers expire
    'risk_free_rate': 0.01,     # 1% annual risk-free rate
    'volatility': 0.3,          # Initial volatility estimate (30%)
    'position_limits': {        # Position limits for each product
        'VOLCANIC_ROCK': 400,
        'VOLCANIC_ROCK_VOUCHER_9500': 200,
        'VOLCANIC_ROCK_VOUCHER_9750': 200,
        'VOLCANIC_ROCK_VOUCHER_10000': 200,
        'VOLCANIC_ROCK_VOUCHER_10250': 200,
        'VOLCANIC_ROCK_VOUCHER_10500': 200,
    },
    'market_making_spread': 0.02,  # Target 2% spread for market making
    'mispricing_threshold': 0.02,  # 2% threshold for arbitrage opportunities
    'max_position_pct': 0.7,       # Maximum % of limit to use for any strategy
    'delta_hedge_threshold': 0.1,  # Threshold for delta hedging
    'delta_rebalance_interval': 100,  # Rebalance hedges every 100 timestamp units
    'gamma_scalping_threshold': 0.1   # Threshold for gamma scalping
}

# Extract strike prices from voucher names
def extract_strike_prices(product_names):
    """
    Extract strike prices from voucher product names
    
    Args:
        product_names: List of product names
        
    Returns:
        Dictionary mapping voucher names to strike prices
    """
    voucher_strikes = {}
    for product in product_names:
        if product.startswith('VOLCANIC_ROCK_VOUCHER_'):
            try:
                strike = int(product.split('_')[-1])
                voucher_strikes[product] = strike
            except ValueError:
                continue
    return voucher_strikes

In [16]:
## 1. Data Loading and Preprocessing

def load_data(filepath):
    """
    Load and preprocess price data
    
    Args:
        filepath: Path to the CSV file
        
    Returns:
        Preprocessed DataFrame
    """
    try:
        # Try to load with standard encoding
        df = pd.read_csv(filepath, sep=';')
    except UnicodeDecodeError:
        # Try with different encoding if standard fails
        df = pd.read_csv(filepath, sep=';', encoding='latin1')
    
    # Convert empty strings to NaN and numeric columns to numbers
    for col in df.columns:
        if col in ['day', 'timestamp']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        elif col not in ['product']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Fill missing values in price columns with NaN
    price_cols = [col for col in df.columns if 'price' in col or 'volume' in col]
    df[price_cols] = df[price_cols].replace('', np.nan)
    
    # Convert to numeric (again to catch any newly exposed issues)
    for col in price_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Filter for volcanic-related products if they exist
    volcanic_df = df[df['product'].str.contains('VOLCANIC', na=False)]
    
    # If no volcanic products found, return original df
    if volcanic_df.empty:
        print(f"No volcanic products found in {filepath}")
        return df
    else:
        print(f"Found {volcanic_df.shape[0]} volcanic product entries in {filepath}")
        return volcanic_df

def process_data(df):
    """
    Process the dataframe to create useful derived data structures
    
    Args:
        df: DataFrame with price data
        
    Returns:
        Tuple containing:
        - Dictionary mapping timestamps to data by product
        - List of unique timestamps
        - List of unique products
        - Dictionary mapping voucher names to strike prices
    """
    # Get unique timestamps and products
    timestamps = sorted(df['timestamp'].unique())
    products = sorted(df['product'].unique())
    
    # Extract voucher strike prices
    strike_prices = extract_strike_prices(products)
    
    # Create a dictionary of data indexed by timestamp and product
    data_by_time = {}
    for ts in timestamps:
        ts_data = df[df['timestamp'] == ts]
        data_by_time[ts] = {}
        
        for _, row in ts_data.iterrows():
            product = row['product']
            data_by_time[ts][product] = row.to_dict()
    
    return data_by_time, timestamps, products, strike_prices

In [17]:
## 2. Option Pricing Models

def black_scholes_call_price(S, K, T, r, sigma):
    """
    Calculate call option price using Black-Scholes formula
    
    Args:
        S: Current price of underlying asset
        K: Strike price
        T: Time to expiration (in years)
        r: Risk-free interest rate
        sigma: Volatility
        
    Returns:
        Call option price
    """
    # Handle special case when very close to expiration
    if T <= 0.001:
        return max(0, S - K)
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

def black_scholes_call_delta(S, K, T, r, sigma):
    """
    Calculate delta of a call option using Black-Scholes formula
    
    Args:
        S: Current price of underlying asset
        K: Strike price
        T: Time to expiration (in years)
        r: Risk-free interest rate
        sigma: Volatility
        
    Returns:
        Call option delta
    """
    # Handle special case when very close to expiration
    if T <= 0.001:
        return 1.0 if S > K else 0.0
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    delta = norm.cdf(d1)
    return delta

def black_scholes_call_gamma(S, K, T, r, sigma):
    """
    Calculate gamma of a call option using Black-Scholes formula
    
    Args:
        S: Current price of underlying asset
        K: Strike price
        T: Time to expiration (in years)
        r: Risk-free interest rate
        sigma: Volatility
        
    Returns:
        Call option gamma
    """
    # Handle special case when very close to expiration
    if T <= 0.001:
        return 0.0
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    return gamma

def black_scholes_call_theta(S, K, T, r, sigma):
    """
    Calculate theta of a call option using Black-Scholes formula
    
    Args:
        S: Current price of underlying asset
        K: Strike price
        T: Time to expiration (in years)
        r: Risk-free interest rate
        sigma: Volatility
        
    Returns:
        Call option theta (per year)
    """
    # Handle special case when very close to expiration
    if T <= 0.001:
        return 0.0
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    theta = -S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)
    return theta

def estimate_implied_volatility(option_price, S, K, T, r, precision=0.001, max_iterations=100):
    """
    Estimate implied volatility using bisection method
    
    Args:
        option_price: Market price of the option
        S: Current price of underlying asset
        K: Strike price
        T: Time to expiration (in years)
        r: Risk-free interest rate
        precision: Desired precision
        max_iterations: Maximum number of iterations
        
    Returns:
        Implied volatility
    """
    if option_price <= 0:
        return 0.001  # Minimum volatility
        
    # Initial guesses
    sigma_low = 0.001
    sigma_high = 5.0
    
    # Check if market price is outside theoretical bounds
    intrinsic_value = max(0, S - K * np.exp(-r * T))
    if option_price < intrinsic_value:
        return 0.001  # Return minimum vol if below intrinsic value
    
    # Bisection search
    for _ in range(max_iterations):
        sigma_mid = (sigma_low + sigma_high) / 2
        price_mid = black_scholes_call_price(S, K, T, r, sigma_mid)
        
        if abs(price_mid - option_price) < precision:
            return sigma_mid
        
        if price_mid > option_price:
            sigma_high = sigma_mid
        else:
            sigma_low = sigma_mid
    
    return (sigma_low + sigma_high) / 2

def calculate_all_greeks(data_by_time, timestamps, products, strike_prices, config):
    """
    Calculate theoretical prices and Greeks for all vouchers at all timestamps
    
    Args:
        data_by_time: Dictionary mapping timestamps to data by product
        timestamps: List of unique timestamps
        products: List of unique products
        strike_prices: Dictionary mapping voucher names to strike prices
        config: Configuration parameters
        
    Returns:
        Dictionary with theoretical prices and Greeks for each voucher at each timestamp
    """
    # Initialize storage for results
    theoretical_data = {}
    
    # Track days elapsed for expiry calculation
    days_elapsed = 0
    prev_day = None
    
    for ts in timestamps:
        theoretical_data[ts] = {}
        
        # Check if this is a new day and update days to expiry
        if ts in data_by_time and any(data_by_time[ts].values()):
            # Get a sample product to check the day
            sample_product = list(data_by_time[ts].keys())[0]
            current_day = data_by_time[ts][sample_product].get('day', prev_day)
            
            if prev_day is not None and current_day != prev_day:
                days_elapsed += 1
            
            prev_day = current_day
        
        # Skip if VOLCANIC_ROCK is not available at this timestamp
        if 'VOLCANIC_ROCK' not in data_by_time.get(ts, {}):
            continue
        
        # Get the underlying price
        rock_data = data_by_time[ts]['VOLCANIC_ROCK']
        rock_price = rock_data['mid_price']
        
        # Time to expiry in years
        T = max(0, (config['days_to_expiry'] - days_elapsed) / 365.0)
        
        # Calculate theoretical prices and Greeks for each voucher
        for voucher, strike in strike_prices.items():
            # Skip if voucher data is not available at this timestamp
            if voucher not in data_by_time.get(ts, {}):
                continue
            
            # Extract market price
            voucher_data = data_by_time[ts][voucher]
            market_price = voucher_data['mid_price']
            
            # Calculate theoretical price and Greeks
            theoretical_price = black_scholes_call_price(
                rock_price, strike, T, config['risk_free_rate'], config['volatility'])
            
            delta = black_scholes_call_delta(
                rock_price, strike, T, config['risk_free_rate'], config['volatility'])
            
            gamma = black_scholes_call_gamma(
                rock_price, strike, T, config['risk_free_rate'], config['volatility'])
            
            theta = black_scholes_call_theta(
                rock_price, strike, T, config['risk_free_rate'], config['volatility'])
            
            # Try to calculate implied volatility
            try:
                implied_vol = estimate_implied_volatility(
                    market_price, rock_price, strike, T, config['risk_free_rate'])
            except:
                implied_vol = config['volatility']  # Use default if calculation fails
            
            # Store results
            theoretical_data[ts][voucher] = {
                'theoretical_price': theoretical_price,
                'market_price': market_price,
                'delta': delta,
                'gamma': gamma,
                'theta': theta,
                'implied_volatility': implied_vol,
                'time_to_expiry': T * 365,  # Store in days
                'mispricing': market_price - theoretical_price,
                'mispricing_pct': (market_price / theoretical_price - 1) * 100 if theoretical_price > 0 else 0
            }
    
    return theoretical_data

def analyze_pricing_accuracy(theoretical_data):
    """
    Analyze accuracy of theoretical pricing model
    
    Args:
        theoretical_data: Dictionary with theoretical prices and Greeks
        
    Returns:
        DataFrame with mispricing statistics
    """
    mispricing_data = []
    
    for ts in theoretical_data:
        for voucher in theoretical_data[ts]:
            data = theoretical_data[ts][voucher]
            mispricing_data.append({
                'timestamp': ts,
                'voucher': voucher,
                'market_price': data['market_price'],
                'theoretical_price': data['theoretical_price'],
                'abs_mispricing': abs(data['mispricing']),
                'mispricing_pct': data['mispricing_pct'],
                'implied_vol': data['implied_volatility'],
                'days_to_expiry': data['time_to_expiry']
            })
    
    mispricing_df = pd.DataFrame(mispricing_data)
    
    # Calculate summary statistics
    stats = mispricing_df.groupby('voucher').agg({
        'abs_mispricing': ['mean', 'std', 'max'],
        'mispricing_pct': ['mean', 'std', 'max'],
        'implied_vol': 'mean'
    })
    
    return mispricing_df, stats

In [18]:
# 3. Market Making Strategy

class MarketMakingStrategy:
    """Market making strategy for volcanic products"""
    
    def __init__(self, config):
        """
        Initialize market making strategy
        
        Args:
            config: Configuration parameters
        """
        self.config = config
        self.positions = {product: 0 for product in config['position_limits']}
        self.cash = 0
        self.trades = []
        self.active_orders = {}
    
    def generate_orders(self, ts, data_by_time, theoretical_data):
        """
        Generate market making orders
        
        Args:
            ts: Current timestamp
            data_by_time: Dictionary mapping timestamps to data by product
            theoretical_data: Dictionary with theoretical prices and Greeks
            
        Returns:
            Dictionary of orders to place (product -> list of (price, quantity) tuples)
        """
        if ts not in data_by_time:
            return {}
        
        orders = {}
        
        for product in data_by_time[ts]:
            # Skip if product not in our target list
            if product not in self.config['position_limits']:
                continue
            
            current_data = data_by_time[ts][product]
            
            # Skip if missing key data
            if 'bid_price_1' not in current_data or 'ask_price_1' not in current_data:
                continue
            
            # Determine fair value
            if product == 'VOLCANIC_ROCK':
                # For the underlying, use mid price
                fair_value = current_data['mid_price']
            elif product in theoretical_data.get(ts, {}):
                # For vouchers, use theoretical price if available
                fair_value = theoretical_data[ts][product]['theoretical_price']
            else:
                # Otherwise use mid price
                fair_value = current_data['mid_price']
            
            # Calculate our bid/ask spread based on product type
            if product == 'VOLCANIC_ROCK':
                # Tighter spread for the underlying
                spread_pct = self.config['market_making_spread'] * 0.5
            else:
                # Wider spread for options, especially far from ATM
                theo_data = theoretical_data.get(ts, {}).get(product, {})
                delta = theo_data.get('delta', 0.5)
                
                # Adjust spread based on delta (wider for far OTM/ITM)
                delta_factor = 1 + 2 * abs(delta - 0.5)  # 1.0 at delta=0.5, up to 2.0 at delta=0/1
                spread_pct = self.config['market_making_spread'] * delta_factor
            
            # Calculate our bid/ask prices
            our_bid = fair_value * (1 - spread_pct)
            our_ask = fair_value * (1 + spread_pct)
            
            # Calculate appropriate quantity based on position limits
            current_position = self.positions.get(product, 0)
            max_position = self.config['position_limits'][product]
            
            # Limit our exposure based on max_position_pct
            available_to_buy = max(0, int((max_position - current_position) * self.config['max_position_pct']))
            available_to_sell = max(0, int((max_position + current_position) * self.config['max_position_pct']))
            
            # Adjust quantity based on price level and available liquidity
            if 'bid_volume_1' in current_data and current_data['bid_volume_1']:
                typical_volume = current_data['bid_volume_1']
                order_size = min(max(1, int(typical_volume * 0.3)), 20)  # 30% of market size, max 20
                
                buy_quantity = min(order_size, available_to_buy)
                sell_quantity = min(order_size, available_to_sell)
            else:
                # Default if no volume data available
                buy_quantity = min(5, available_to_buy)
                sell_quantity = min(5, available_to_sell)
            
            # Check if our bid is better than market
            if current_data['bid_price_1'] and our_bid > current_data['bid_price_1'] and buy_quantity > 0:
                # Place a buy order
                if product not in orders:
                    orders[product] = []
                orders[product].append((our_bid, buy_quantity))
            
            # Check if our ask is better than market
            if current_data['ask_price_1'] and our_ask < current_data['ask_price_1'] and sell_quantity > 0:
                # Place a sell order
                if product not in orders:
                    orders[product] = []
                orders[product].append((our_ask, -sell_quantity))  # Negative for sell
        
        return orders
    
    def execute_orders(self, ts, orders, data_by_time):
        """
        Simulate order execution
        
        Args:
            ts: Current timestamp
            orders: Dictionary of orders (product -> list of (price, quantity) tuples)
            data_by_time: Dictionary mapping timestamps to data by product
            
        Returns:
            List of executed trades
        """
        executed_trades = []
        
        for product, product_orders in orders.items():
            if ts not in data_by_time or product not in data_by_time[ts]:
                continue
            
            market_data = data_by_time[ts][product]
            
            for price, quantity in product_orders:
                # Determine if this is a buy or sell order
                is_buy = quantity > 0
                
                # Check if our order would match with the market
                if is_buy and market_data['ask_price_1'] and price >= market_data['ask_price_1']:
                    # Our buy order matches with market's best ask
                    exec_price = market_data['ask_price_1']
                    exec_quantity = min(abs(quantity), market_data['ask_volume_1'])
                elif not is_buy and market_data['bid_price_1'] and price <= market_data['bid_price_1']:
                    # Our sell order matches with market's best bid
                    exec_price = market_data['bid_price_1']
                    exec_quantity = min(abs(quantity), market_data['bid_volume_1'])
                else:
                    # No immediate execution
                    exec_quantity = 0
                
                if exec_quantity > 0:
                    # Record the trade
                    direction = 1 if is_buy else -1
                    executed_trades.append({
                        'timestamp': ts,
                        'product': product,
                        'price': exec_price,
                        'quantity': exec_quantity * direction,
                        'strategy': 'market_making'
                    })
                    
                    # Update position and cash
                    self.positions[product] += exec_quantity * direction
                    self.cash -= exec_price * exec_quantity * direction
        
        # Update trade history
        self.trades.extend(executed_trades)
        
        return executed_trades
    
    def calculate_pnl(self, final_prices):
        """
        Calculate final P&L
        
        Args:
            final_prices: Dictionary mapping products to final prices
            
        Returns:
            Dictionary with P&L breakdown
        """
        # Calculate P&L from realized trades
        realized_pnl = sum(trade['price'] * -trade['quantity'] for trade in self.trades)
        
        # Calculate P&L from open positions
        unrealized_pnl = sum(
            self.positions[product] * final_prices.get(product, 0)
            for product in self.positions
        )
        
        total_pnl = realized_pnl + unrealized_pnl + self.cash
        
        return {
            'realized_pnl': realized_pnl,
            'unrealized_pnl': unrealized_pnl,
            'cash': self.cash,
            'total_pnl': total_pnl
        }


In [19]:
## 4. Arbitrage Strategy

class ArbitrageStrategy:
    """Arbitrage strategy for volcanic vouchers"""
    
    def __init__(self, config):
        """
        Initialize arbitrage strategy
        
        Args:
            config: Configuration parameters
        """
        self.config = config
        self.positions = {product: 0 for product in config['position_limits']}
        self.cash = 0
        self.trades = []
    
    def generate_orders(self, ts, data_by_time, theoretical_data):
        """
        Generate arbitrage orders based on mispricing
        
        Args:
            ts: Current timestamp
            data_by_time: Dictionary mapping timestamps to data by product
            theoretical_data: Dictionary with theoretical prices and Greeks
            
        Returns:
            Dictionary of orders to place (product -> list of (price, quantity) tuples)
        """
        if ts not in data_by_time or ts not in theoretical_data:
            return {}
        
        orders = {}
        
        # Find mispriced vouchers
        for product in theoretical_data[ts]:
            # Only consider vouchers
            if product == 'VOLCANIC_ROCK' or product not in data_by_time[ts]:
                continue
            
            theo_data = theoretical_data[ts][product]
            market_data = data_by_time[ts][product]
            
            # Skip if missing key data
            if 'bid_price_1' not in market_data or 'ask_price_1' not in market_data:
                continue
            
            # Calculate mispricing percentage
            theo_price = theo_data['theoretical_price']
            mispricing_pct = theo_data['mispricing_pct']
            
            # Determine trade direction based on mispricing
            if abs(mispricing_pct) > self.config['mispricing_threshold'] * 100:
                # Calculate appropriate quantity based on position limits
                current_position = self.positions.get(product, 0)
                max_position = self.config['position_limits'][product]
                
                if mispricing_pct < 0:
                    # Market price is below theoretical price - BUY
                    available_qty = max(0, int((max_position - current_position) * self.config['max_position_pct']))
                    
                    if available_qty > 0:
                        # Limit order at or slightly above market's best ask
                        order_price = market_data['ask_price_1']
                        order_qty = min(available_qty, market_data['ask_volume_1'])
                        
                        if order_qty > 0:
                            if product not in orders:
                                orders[product] = []
                            orders[product].append((order_price, order_qty))
                else:
                    # Market price is above theoretical price - SELL
                    available_qty = max(0, int((max_position + current_position) * self.config['max_position_pct']))
                    
                    if available_qty > 0:
                        # Limit order at or slightly below market's best bid
                        order_price = market_data['bid_price_1']
                        order_qty = min(available_qty, market_data['bid_volume_1'])
                        
                        if order_qty > 0:
                            if product not in orders:
                                orders[product] = []
                            orders[product].append((order_price, -order_qty))  # Negative for sell
        
        return orders
    
    def execute_orders(self, ts, orders, data_by_time):
        """
        Simulate order execution
        
        Args:
            ts: Current timestamp
            orders: Dictionary of orders (product -> list of (price, quantity) tuples)
            data_by_time: Dictionary mapping timestamps to data by product
            
        Returns:
            List of executed trades
        """
        executed_trades = []
        
        for product, product_orders in orders.items():
            if ts not in data_by_time or product not in data_by_time[ts]:
                continue
            
            market_data = data_by_time[ts][product]
            
            for price, quantity in product_orders:
                # Determine if this is a buy or sell order
                is_buy = quantity > 0
                
                # Check if our order would match with the market
                if is_buy and market_data['ask_price_1'] and price >= market_data['ask_price_1']:
                    # Our buy order matches with market's best ask
                    exec_price = market_data['ask_price_1']
                    exec_quantity = min(abs(quantity), market_data['ask_volume_1'])
                elif not is_buy and market_data['bid_price_1'] and price <= market_data['bid_price_1']:
                    # Our sell order matches with market's best bid
                    exec_price = market_data['bid_price_1']
                    exec_quantity = min(abs(quantity), market_data['bid_volume_1'])
                else:
                    # No immediate execution
                    exec_quantity = 0
                
                if exec_quantity > 0:
                    # Record the trade
                    direction = 1 if is_buy else -1
                    executed_trades.append({
                        'timestamp': ts,
                        'product': product,
                        'price': exec_price,
                        'quantity': exec_quantity * direction,
                        'strategy': 'arbitrage'
                    })
                    
                    # Update position and cash
                    self.positions[product] += exec_quantity * direction
                    self.cash -= exec_price * exec_quantity * direction
        
        # Update trade history
        self.trades.extend(executed_trades)
        
        return executed_trades
    
    def calculate_pnl(self, final_prices):
        """
        Calculate final P&L
        
        Args:
            final_prices: Dictionary mapping products to final prices
            
        Returns:
            Dictionary with P&L breakdown
        """
        # Calculate P&L from realized trades
        realized_pnl = sum(trade['price'] * -trade['quantity'] for trade in self.trades)
        
        # Calculate P&L from open positions
        unrealized_pnl = sum(
            self.positions[product] * final_prices.get(product, 0)
            for product in self.positions
        )
        
        total_pnl = realized_pnl + unrealized_pnl + self.cash
        
        return {
            'realized_pnl': realized_pnl,
            'unrealized_pnl': unrealized_pnl,
            'cash': self.cash,
            'total_pnl': total_pnl
        }


In [20]:
## 5. Delta Hedging Strategy

class DeltaHedgingStrategy:
    """Delta hedging strategy for volcanic vouchers"""
    
    def __init__(self, config):
        """
        Initialize delta hedging strategy
        
        Args:
            config: Configuration parameters
        """
        self.config = config
        self.positions = {product: 0 for product in config['position_limits']}
        self.cash = 0
        self.trades = []
        self.last_rebalance_ts = None
    
    def calculate_portfolio_delta(self, ts, theoretical_data):
        """
        Calculate the delta of the entire portfolio
        
        Args:
            ts: Current timestamp
            theoretical_data: Dictionary with theoretical prices and Greeks
            
        Returns:
            Net delta of the portfolio
        """
        if ts not in theoretical_data:
            return 0
        
        # Start with position in the underlying (delta = 1.0)
        net_delta = self.positions.get('VOLCANIC_ROCK', 0)
        
        # Add delta from voucher positions
        for product in theoretical_data[ts]:
            if product == 'VOLCANIC_ROCK':
                continue
            
            position = self.positions.get(product, 0)
            if position != 0:
                delta = theoretical_data[ts][product]['delta']
                net_delta += position * delta
        
        return net_delta
    
    def generate_orders(self, ts, data_by_time, theoretical_data):
        """
        Generate orders to maintain delta-neutral portfolio
        
        Args:
            ts: Current timestamp
            data_by_time: Dictionary mapping timestamps to data by product
            theoretical_data: Dictionary with theoretical prices and Greeks
            
        Returns:
            Dictionary of orders to place (product -> list of (price, quantity) tuples)
        """
        # Only rebalance at specified intervals
        if (self.last_rebalance_ts is not None and 
            ts - self.last_rebalance_ts < self.config['delta_rebalance_interval']):
            return {}
        
        if ts not in data_by_time or 'VOLCANIC_ROCK' not in data_by_time[ts]:
            return {}
        
        # Calculate current portfolio delta
        net_delta = self.calculate_portfolio_delta(ts, theoretical_data)
        
        # If delta exposure is within threshold, no need to hedge
        if abs(net_delta) <= self.config['delta_hedge_threshold'] * self.config['position_limits']['VOLCANIC_ROCK']:
            return {}
        
        # Calculate target position in the underlying to offset delta
        current_position = self.positions.get('VOLCANIC_ROCK', 0)
        target_position = -int(net_delta)  # Round to nearest integer
        position_change = target_position - current_position
        
        # Respect position limits
        max_position = self.config['position_limits']['VOLCANIC_ROCK']
        if target_position > max_position:
            position_change = max_position - current_position
        elif target_position < -max_position:
            position_change = -max_position - current_position
        
        # If position change is too small, ignore
        if abs(position_change) < 1:
            return {}
        
        # Generate order for the underlying
        orders = {}
        market_data = data_by_time[ts]['VOLCANIC_ROCK']
        
        if position_change > 0:
            # Need to buy the underlying
            order_price = market_data['ask_price_1']
            order_qty = min(position_change, market_data['ask_volume_1'])
            
            if order_qty > 0:
                orders['VOLCANIC_ROCK'] = [(order_price, order_qty)]
        else:
            # Need to sell the underlying
            order_price = market_data['bid_price_1']
            order_qty = min(abs(position_change), market_data['bid_volume_1'])
            
            if order_qty > 0:
                orders['VOLCANIC_ROCK'] = [(order_price, -order_qty)]
        
        # Update last rebalance timestamp
        self.last_rebalance_ts = ts
        
        return orders
    
    def execute_orders(self, ts, orders, data_by_time):
        """
        Simulate order execution
        
        Args:
            ts: Current timestamp
            orders: Dictionary of orders (product -> list of (price, quantity) tuples)
            data_by_time: Dictionary mapping timestamps to data by product
            
        Returns:
            List of executed trades
        """
        executed_trades = []
        
        for product, product_orders in orders.items():
            if ts not in data_by_time or product not in data_by_time[ts]:
                continue
            
            market_data = data_by_time[ts][product]
            
            for price, quantity in product_orders:
                # Determine if this is a buy or sell order
                is_buy = quantity > 0
                
                # Check if our order would match with the market
                if is_buy and market_data['ask_price_1'] and price >= market_data['ask_price_1']:
                    # Our buy order matches with market's best ask
                    exec_price = market_data['ask_price_1']
                    exec_quantity = min(abs(quantity), market_data['ask_volume_1'])
                elif not is_buy and market_data['bid_price_1'] and price <= market_data['bid_price_1']:
                    # Our sell order matches with market's best bid
                    exec_price = market_data['bid_price_1']
                    exec_quantity = min(abs(quantity), market_data['bid_volume_1'])
                else:
                    # No immediate execution
                    exec_quantity = 0
                
                if exec_quantity > 0:
                    # Record the trade
                    direction = 1 if is_buy else -1
                    executed_trades.append({
                        'timestamp': ts,
                        'product': product,
                        'price': exec_price,
                        'quantity': exec_quantity * direction,
                        'strategy': 'delta_hedging'
                    })
                    
                    # Update position and cash
                    self.positions[product] += exec_quantity * direction
                    self.cash -= exec_price * exec_quantity * direction
        
        # Update trade history
        self.trades.extend(executed_trades)
        
        return executed_trades
    
    def calculate_pnl(self, final_prices):
        """
        Calculate final P&L
        
        Args:
            final_prices: Dictionary mapping products to final prices
            
        Returns:
            Dictionary with P&L breakdown
        """
        # Calculate P&L from realized trades
        realized_pnl = sum(trade['price'] * -trade['quantity'] for trade in self.trades)
        
        # Calculate P&L from open positions
        unrealized_pnl = sum(
            self.positions[product] * final_prices.get(product, 0)
            for product in self.positions
        )
        
        total_pnl = realized_pnl + unrealized_pnl + self.cash
        
        return {
            'realized_pnl': realized_pnl,
            'unrealized_pnl': unrealized_pnl,
            'cash': self.cash,
            'total_pnl': total_pnl
        }


In [21]:
## 6. Gamma Scalping Strategy

class GammaScalpingStrategy:
    """Gamma scalping strategy for volcanic vouchers"""
    
    def __init__(self, config):
        """
        Initialize gamma scalping strategy
        
        Args:
            config: Configuration parameters
        """
        self.config = config
        self.positions = {product: 0 for product in config['position_limits']}
        self.cash = 0
        self.trades = []
        self.last_rebalance_ts = None
        self.last_rock_price = None
    
    def generate_orders(self, ts, data_by_time, theoretical_data):
        """
        Generate orders for gamma scalping
        
        Args:
            ts: Current timestamp
            data_by_time: Dictionary mapping timestamps to data by product
            theoretical_data: Dictionary with theoretical prices and Greeks
            
        Returns:
            Dictionary of orders to place (product -> list of (price, quantity) tuples)
        """
        if ts not in data_by_time or 'VOLCANIC_ROCK' not in data_by_time[ts]:
            return {}
        
        # Get current price of the underlying
        current_rock_price = data_by_time[ts]['VOLCANIC_ROCK']['mid_price']
        
        # If this is the first observation, just store price and return
        if self.last_rock_price is None:
            self.last_rock_price = current_rock_price
            return {}
        
        # Calculate price change since last observation
        price_change = current_rock_price - self.last_rock_price
        
        # Only act if price change is significant (to avoid overtrading)
        if abs(price_change) < self.config['gamma_scalping_threshold'] * current_rock_price:
            return {}
        
        # Find vouchers with high gamma
        high_gamma_vouchers = []
        if ts in theoretical_data:
            for product in theoretical_data[ts]:
                if product != 'VOLCANIC_ROCK':
                    gamma = theoretical_data[ts][product].get('gamma', 0)
                    if gamma > 0.0001:  # Threshold for "high gamma"
                        high_gamma_vouchers.append({
                            'product': product,
                            'gamma': gamma,
                            'delta': theoretical_data[ts][product].get('delta', 0.5)
                        })
        
        # Sort vouchers by gamma (highest first)
        high_gamma_vouchers.sort(key=lambda x: x['gamma'], reverse=True)
        
        orders = {}
        
        # If no high-gamma vouchers or price hasn't changed enough, no action
        if not high_gamma_vouchers:
            return orders
        
        # Focus on the voucher with highest gamma
        target_voucher = high_gamma_vouchers[0]['product']
        target_gamma = high_gamma_vouchers[0]['gamma']
        target_delta = high_gamma_vouchers[0]['delta']
        
        # Determine optimal position size based on gamma
        # Higher gamma = more profit potential from scalping
        size_factor = min(1.0, target_gamma * 10000)  # Scale factor based on gamma
        max_position = self.config['position_limits'][target_voucher]
        target_size = int(max_position * size_factor * 0.3)  # Use up to 30% of position limit
        
        current_position = self.positions.get(target_voucher, 0)
        
        # If price has gone up, we want to sell vouchers (and vice versa)
        # This is because we've made profit on long voucher positions when price rises
        # and want to lock in gains by selling some
        if price_change > 0 and current_position > 0:
            # Sell some vouchers to lock in gains
            market_data = data_by_time[ts][target_voucher]
            sell_size = min(current_position, target_size)
            
            if sell_size > 0 and 'bid_price_1' in market_data:
                orders[target_voucher] = [(market_data['bid_price_1'], -sell_size)]
        
        elif price_change < 0 and current_position < max_position:
            # Buy vouchers when price drops (they're cheaper now)
            market_data = data_by_time[ts][target_voucher]
            buy_size = min(max_position - current_position, target_size)
            
            if buy_size > 0 and 'ask_price_1' in market_data:
                orders[target_voucher] = [(market_data['ask_price_1'], buy_size)]
        
        # Update last price
        self.last_rock_price = current_rock_price
        
        return orders
    
    def execute_orders(self, ts, orders, data_by_time):
        """
        Simulate order execution
        
        Args:
            ts: Current timestamp
            orders: Dictionary of orders (product -> list of (price, quantity) tuples)
            data_by_time: Dictionary mapping timestamps to data by product
            
        Returns:
            List of executed trades
        """
        executed_trades = []
        
        for product, product_orders in orders.items():
            if ts not in data_by_time or product not in data_by_time[ts]:
                continue
            
            market_data = data_by_time[ts][product]
            
            for price, quantity in product_orders:
                # Determine if this is a buy or sell order
                is_buy = quantity > 0
                
                # Check if our order would match with the market
                if is_buy and market_data['ask_price_1'] and price >= market_data['ask_price_1']:
                    # Our buy order matches with market's best ask
                    exec_price = market_data['ask_price_1']
                    exec_quantity = min(abs(quantity), market_data['ask_volume_1'])
                elif not is_buy and market_data['bid_price_1'] and price <= market_data['bid_price_1']:
                    # Our sell order matches with market's best bid
                    exec_price = market_data['bid_price_1']
                    exec_quantity = min(abs(quantity), market_data['bid_volume_1'])
                else:
                    # No immediate execution
                    exec_quantity = 0
                
                if exec_quantity > 0:
                    # Record the trade
                    direction = 1 if is_buy else -1
                    executed_trades.append({
                        'timestamp': ts,
                        'product': product,
                        'price': exec_price,
                        'quantity': exec_quantity * direction,
                        'strategy': 'gamma_scalping'
                    })
                    
                    # Update position and cash
                    self.positions[product] += exec_quantity * direction
                    self.cash -= exec_price * exec_quantity * direction
        
        # Update trade history
        self.trades.extend(executed_trades)
        
        return executed_trades
    
    def calculate_pnl(self, final_prices):
        """
        Calculate final P&L
        
        Args:
            final_prices: Dictionary mapping products to final prices
            
        Returns:
            Dictionary with P&L breakdown
        """
        # Calculate P&L from realized trades
        realized_pnl = sum(trade['price'] * -trade['quantity'] for trade in self.trades)
        
        # Calculate P&L from open positions
        unrealized_pnl = sum(
            self.positions[product] * final_prices.get(product, 0)
            for product in self.positions
        )
        
        total_pnl = realized_pnl + unrealized_pnl + self.cash
        
        return {
            'realized_pnl': realized_pnl,
            'unrealized_pnl': unrealized_pnl,
            'cash': self.cash,
            'total_pnl': total_pnl
        }


In [22]:
## 7. Combined Trading Strategy

class CombinedStrategy:
    """Combined trading strategy that uses all individual strategies"""
    
    def __init__(self, config):
        """
        Initialize combined strategy
        
        Args:
            config: Configuration parameters
        """
        self.config = config
        self.market_making = MarketMakingStrategy(config)
        self.arbitrage = ArbitrageStrategy(config)
        self.delta_hedging = DeltaHedgingStrategy(config)
        self.gamma_scalping = GammaScalpingStrategy(config)
        
        self.positions = {product: 0 for product in config['position_limits']}
        self.cash = 0
        self.trades = []
        
        self.strategy_weights = {
            'market_making': 0.4,
            'arbitrage': 0.3,
            'delta_hedging': 0.2,
            'gamma_scalping': 0.1
        }
    
    def generate_and_execute_orders(self, ts, data_by_time, theoretical_data):
        """
        Generate and execute orders from all strategies
        
        Args:
            ts: Current timestamp
            data_by_time: Dictionary mapping timestamps to data by product
            theoretical_data: Dictionary with theoretical prices and Greeks
            
        Returns:
            List of executed trades
        """
        executed_trades = []
        
        # Market making strategy
        mm_orders = self.market_making.generate_orders(ts, data_by_time, theoretical_data)
        mm_trades = self.market_making.execute_orders(ts, mm_orders, data_by_time)
        executed_trades.extend(mm_trades)
        
        # Arbitrage strategy
        arb_orders = self.arbitrage.generate_orders(ts, data_by_time, theoretical_data)
        arb_trades = self.arbitrage.execute_orders(ts, arb_orders, data_by_time)
        executed_trades.extend(arb_trades)
        
        # Delta hedging strategy
        dh_orders = self.delta_hedging.generate_orders(ts, data_by_time, theoretical_data)
        dh_trades = self.delta_hedging.execute_orders(ts, dh_orders, data_by_time)
        executed_trades.extend(dh_trades)
        
        # Gamma scalping strategy
        gs_orders = self.gamma_scalping.generate_orders(ts, data_by_time, theoretical_data)
        gs_trades = self.gamma_scalping.execute_orders(ts, gs_orders, data_by_time)
        executed_trades.extend(gs_trades)
        
        # Update combined positions and cash
        # Reset positions first
        for product in self.positions:
            self.positions[product] = 0
        
        # Sum positions from all strategies
        for strategy in [self.market_making, self.arbitrage, self.delta_hedging, self.gamma_scalping]:
            for product, position in strategy.positions.items():
                if product in self.positions:
                    self.positions[product] += position
        
        # Track trades
        self.trades.extend(executed_trades)
        
        return executed_trades
    
    def calculate_pnl(self, final_prices):
        """
        Calculate final P&L
        
        Args:
            final_prices: Dictionary mapping products to final prices
            
        Returns:
            Dictionary with P&L breakdown
        """
        # Calculate P&L for each strategy
        mm_pnl = self.market_making.calculate_pnl(final_prices)
        arb_pnl = self.arbitrage.calculate_pnl(final_prices)
        dh_pnl = self.delta_hedging.calculate_pnl(final_prices)
        gs_pnl = self.gamma_scalping.calculate_pnl(final_prices)
        
        # Sum up P&L from all strategies
        total_pnl = (
            mm_pnl['total_pnl'] +
            arb_pnl['total_pnl'] +
            dh_pnl['total_pnl'] +
            gs_pnl['total_pnl']
        )
        
        return {
            'market_making_pnl': mm_pnl['total_pnl'],
            'arbitrage_pnl': arb_pnl['total_pnl'],
            'delta_hedging_pnl': dh_pnl['total_pnl'],
            'gamma_scalping_pnl': gs_pnl['total_pnl'],
            'total_pnl': total_pnl
        }


In [23]:
## 8. Backtesting Framework

def run_backtest(data_by_time, timestamps, products, strike_prices, config):
    """
    Run a backtest of the trading strategy
    
    Args:
        data_by_time: Dictionary mapping timestamps to data by product
        timestamps: List of unique timestamps
        products: List of unique products
        strike_prices: Dictionary mapping voucher names to strike prices
        config: Configuration parameters
        
    Returns:
        Dictionary with backtest results
    """
    # Calculate theoretical prices and Greeks
    theoretical_data = calculate_all_greeks(data_by_time, timestamps, products, strike_prices, config)
    
    # Initialize the combined strategy
    strategy = CombinedStrategy(config)
    
    # Lists to track results
    executed_trades = []
    portfolio_values = []
    position_history = []
    
    # Run the simulation
    for ts in timestamps:
        # Generate and execute orders
        trades = strategy.generate_and_execute_orders(ts, data_by_time, theoretical_data)
        executed_trades.extend(trades)
        
        # Record positions
        position_history.append({
            'timestamp': ts,
            'positions': strategy.positions.copy()
        })
        
        # Calculate portfolio value
        if ts in data_by_time:
            portfolio_value = strategy.cash
            for product, position in strategy.positions.items():
                if position != 0 and product in data_by_time[ts]:
                    portfolio_value += position * data_by_time[ts][product]['mid_price']
            
            portfolio_values.append({
                'timestamp': ts,
                'value': portfolio_value
            })
    
    # Get final prices
    final_prices = {}
    last_ts = timestamps[-1]
    for product in products:
        if product in data_by_time.get(last_ts, {}):
            final_prices[product] = data_by_time[last_ts][product]['mid_price']
    
    # Calculate final P&L
    pnl = strategy.calculate_pnl(final_prices)
    
    return {
        'trades': executed_trades,
        'portfolio_values': portfolio_values,
        'position_history': position_history,
        'pnl': pnl,
        'final_positions': strategy.positions,
        'theoretical_data': theoretical_data
    }

def analyze_results(results, timestamps, products):
    """
    Analyze backtest results
    
    Args:
        results: Dictionary with backtest results
        timestamps: List of unique timestamps
        products: List of unique products
        
    Returns:
        None (creates visualizations and prints statistics)
    """
    # Extract results
    trades = results['trades']
    portfolio_values = results['portfolio_values']
    position_history = results['position_history']
    pnl = results['pnl']
    
    # Convert trades to DataFrame
    trades_df = pd.DataFrame(trades)
    
    # Convert portfolio values to DataFrame
    portfolio_df = pd.DataFrame(portfolio_values)
    
    # Plot portfolio value over time
    plt.figure(figsize=(12, 6))
    plt.plot(portfolio_df['timestamp'], portfolio_df['value'])
    plt.title('Portfolio Value Over Time')
    plt.xlabel('Timestamp')
    plt.ylabel('Value')
    plt.grid(True)
    plt.show()
    
    # Plot P&L by strategy
    strategy_pnl = {
        'Market Making': pnl['market_making_pnl'],
        'Arbitrage': pnl['arbitrage_pnl'],
        'Delta Hedging': pnl['delta_hedging_pnl'],
        'Gamma Scalping': pnl['gamma_scalping_pnl']
    }
    
    plt.figure(figsize=(10, 6))
    plt.bar(strategy_pnl.keys(), strategy_pnl.values())
    plt.title('P&L by Strategy')
    plt.ylabel('P&L')
    plt.grid(True, axis='y')
    plt.show()
    
    # Plot positions over time for each product
    position_data = {}
    for item in position_history:
        ts = item['timestamp']
        for product, position in item['positions'].items():
            if product not in position_data:
                position_data[product] = []
            position_data[product].append((ts, position))
    
    plt.figure(figsize=(14, 8))
    for product in products:
        if product in position_data and any(pos[1] != 0 for pos in position_data[product]):
            x = [pos[0] for pos in position_data[product]]
            y = [pos[1] for pos in position_data[product]]
            plt.plot(x, y, label=product)
    
    plt.title('Positions Over Time')
    plt.xlabel('Timestamp')
    plt.ylabel('Position')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Calculate trade statistics
    if not trades_df.empty:
        trade_count = len(trades_df)
        trade_volume = trades_df['quantity'].abs().sum()
        product_count = trades_df['product'].nunique()
        
        print(f"Total trades: {trade_count}")
        print(f"Total volume: {trade_volume}")
        print(f"Products traded: {product_count}")
        
        # Trades by strategy
        strategy_counts = trades_df['strategy'].value_counts()
        print("\nTrades by strategy:")
        for strategy, count in strategy_counts.items():
            print(f"  {strategy}: {count} trades")
        
        # Trades by product
        product_counts = trades_df['product'].value_counts()
        print("\nTrades by product:")
        for product, count in product_counts.items():
            print(f"  {product}: {count} trades")
    
    # Print final P&L
    print("\nFinal P&L:")
    print(f"  Market Making: {pnl['market_making_pnl']:.2f}")
    print(f"  Arbitrage: {pnl['arbitrage_pnl']:.2f}")
    print(f"  Delta Hedging: {pnl['delta_hedging_pnl']:.2f}")
    print(f"  Gamma Scalping: {pnl['gamma_scalping_pnl']:.2f}")
    print(f"  Total: {pnl['total_pnl']:.2f}")


In [24]:
## 9. Main function to run the backtest

def main():
    """
    Main function to run the backtest
    """
    # Define configuration
    config = CONFIG
    
    # Load and process data
    # Modify the file path as needed
    file_path = 'data/prices_round_3_day_0.csv'
    df = load_data(file_path)
    
    # Print sample data
    print("Sample data:")
    print(df.head())
    
    # Process data
    data_by_time, timestamps, products, strike_prices = process_data(df)
    
    # Print information about the data
    print(f"Loaded {len(timestamps)} timestamps and {len(products)} products")
    print(f"Strike prices: {strike_prices}")
    
    # Run backtest
    print("Running backtest...")
    results = run_backtest(data_by_time, timestamps, products, strike_prices, config)
    
    # Analyze results
    print("Analyzing results...")
    analyze_results(results, timestamps, products)
    
    # Return results for further analysis
    return results, data_by_time, timestamps, products, strike_prices

if __name__ == "__main__":
    results, data_by_time, timestamps, products, strike_prices = main()


Found 60000 volcanic product entries in data/prices_round_3_day_0.csv
Sample data:
    day  timestamp                      product  bid_price_1  bid_volume_1  \
0     0          0  VOLCANIC_ROCK_VOUCHER_10500         99.0          19.0   
4     0          0  VOLCANIC_ROCK_VOUCHER_10000        505.0          19.0   
6     0          0   VOLCANIC_ROCK_VOUCHER_9750        754.0          19.0   
9     0          0   VOLCANIC_ROCK_VOUCHER_9500       1003.0          19.0   
11    0          0  VOLCANIC_ROCK_VOUCHER_10250        273.0          19.0   

    bid_price_2  bid_volume_2  bid_price_3  bid_volume_3  ask_price_1  \
0           NaN           NaN          NaN           NaN          100   
4           NaN           NaN          NaN           NaN          506   
6           NaN           NaN          NaN           NaN          755   
9           NaN           NaN          NaN           NaN         1004   
11          NaN           NaN          NaN           NaN          274   

    ask_v

ValueError: cannot convert float NaN to integer

In [ ]:
## 10. Strategy Tuning and Optimization

def optimize_parameters(data_by_time, timestamps, products, strike_prices, param_ranges):
    """
    Optimize strategy parameters
    
    Args:
        data_by_time: Dictionary mapping timestamps to data by product
        timestamps: List of unique timestamps
        products: List of unique products
        strike_prices: Dictionary mapping voucher names to strike prices
        param_ranges: Dictionary mapping parameter names to lists of values to test
        
    Returns:
        Dictionary with optimal parameters and performance results
    """
    best_pnl = float('-inf')
    best_params = None
    all_results = []
    
    # Generate all parameter combinations
    import itertools
    param_names = list(param_ranges.keys())
    param_values = list(param_ranges.values())
    param_combinations = list(itertools.product(*param_values))
    
    print(f"Testing {len(param_combinations)} parameter combinations...")
    
    for combination in param_combinations:
        # Create config with this parameter combination
        test_config = CONFIG.copy()
        for i, param_name in enumerate(param_names):
            param_path = param_name.split('.')
            
            if len(param_path) == 1:
                test_config[param_path[0]] = combination[i]
            elif len(param_path) == 2:
                if param_path[0] not in test_config:
                    test_config[param_path[0]] = {}
                test_config[param_path[0]][param_path[1]] = combination[i]
        
        # Create a subset of data for faster testing
        # Use every 10th timestamp
        test_timestamps = timestamps[::10]
        
        # Run backtest with these parameters
        results = run_backtest(data_by_time, test_timestamps, products, strike_prices, test_config)
        
        # Record results
        param_dict = {param_name: combination[i] for i, param_name in enumerate(param_names)}
        total_pnl = results['pnl']['total_pnl']
        
        all_results.append({
            'params': param_dict,
            'pnl': total_pnl
        })
        
        # Update best parameters
        if total_pnl > best_pnl:
            best_pnl = total_pnl
            best_params = param_dict
    
    # Sort results by PnL
    all_results.sort(key=lambda x: x['pnl'], reverse=True)
    
    print("\nTop 5 parameter combinations:")
    for i, result in enumerate(all_results[:5]):
        params_str = ', '.join(f"{k}={v}" for k, v in result['params'].items())
        print(f"{i+1}. PnL: {result['pnl']:.2f}, Params: {params_str}")
    
    print(f"\nBest parameters: {best_params}")
    print(f"Best PnL: {best_pnl:.2f}")
    
    return {
        'best_params': best_params,
        'best_pnl': best_pnl,
        'all_results': all_results
    }

def run_strategy_comparison():
    """
    Compare performance of different strategy combinations
    
    Returns:
        DataFrame with strategy performance comparison
    """
    # Define configuration
    config = CONFIG
    
    # Load and process data
    file_path = 'prices_round_3_day_0.csv'
    df = load_data(file_path)
    data_by_time, timestamps, products, strike_prices = process_data(df)
    
    # Strategy combinations to test
    strategy_combinations = [
        {'name': 'Market Making Only', 'weights': {'market_making': 1.0, 'arbitrage': 0.0, 'delta_hedging': 0.0, 'gamma_scalping': 0.0}},
        {'name': 'Arbitrage Only', 'weights': {'market_making': 0.0, 'arbitrage': 1.0, 'delta_hedging': 0.0, 'gamma_scalping': 0.0}},
        {'name': 'Delta Hedging Only', 'weights': {'market_making': 0.0, 'arbitrage': 0.0, 'delta_hedging': 1.0, 'gamma_scalping': 0.0}},
        {'name': 'Gamma Scalping Only', 'weights': {'market_making': 0.0, 'arbitrage': 0.0, 'delta_hedging': 0.0, 'gamma_scalping': 1.0}},
        {'name': 'MM + Arbitrage', 'weights': {'market_making': 0.5, 'arbitrage': 0.5, 'delta_hedging': 0.0, 'gamma_scalping': 1.0}},